In [27]:

import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN, Dense

In [65]:
data=pd.read_csv(r"C:\Users\Public\Downloads\swiggy.csv")

In [66]:
data

,ID,Area,City,Restaurant Price,Avg Rating,Total Rating,Food Item,Food Type,Delivery Time,Review
0,1,Suburb,Ahmedabad,600,4.2,6198,Sushi,Fast Food,30-40 min,"Good, but nothing extraordinary."
1,2,Business District,Pune,200,4.7,4865,Pepperoni Pizza,Non-Vegetarian,50-60 min,"Good, but nothing extraordinary."
2,3,Suburb,Bangalore,600,4.7,2095,Waffles,Fast Food,50-60 min,Late delivery ruined it.
3,4,Business District,Mumbai,900,4.0,6639,Sushi,Vegetarian,50-60 min,Best meal I've had in a while!
4,5,Tech Park,Mumbai,200,4.7,6926,Spring Rolls,Gluten-Free,20-30 min,Mediocre experience.
...,...,...,...,...,...,...,...,...,...,...
7995,7996,City Center,Mumbai,300,4.0,3303,BBQ Ribs,Vegan,20-30 min,My new favorite dish!
7996,7997,Downtown,Chennai,100,4.7,8742,Butter Chicken,Non-Vegetarian,20-30 min,Amazing taste and quick delivery.
7997,7998,Tech Park,Chennai,900,4.5,4645,Mango Shake,Fast Food,30-40 min,Nothing special but edible.
7998,7999,Old Town,Delhi,500,4.2,3218,Grilled Cheese,Non-Vegetarian,50-60 min,It was okay.


In [67]:
data["Avg Rating"].unique()

array([4.2, 4.7, 4. , 4.5, 3. , 5. , 3.5])

In [25]:
data["Review"].notnull().sum()

8000

In [26]:
data.drop_duplicates()

,ID,Area,City,Restaurant Price,Avg Rating,Total Rating,Food Item,Food Type,Delivery Time,Review
0,1,Suburb,Ahmedabad,600,4.2,6198,Sushi,Fast Food,30-40 min,"Good, but nothing extraordinary."
1,2,Business District,Pune,200,4.7,4865,Pepperoni Pizza,Non-Vegetarian,50-60 min,"Good, but nothing extraordinary."
2,3,Suburb,Bangalore,600,4.7,2095,Waffles,Fast Food,50-60 min,Late delivery ruined it.
3,4,Business District,Mumbai,900,4.0,6639,Sushi,Vegetarian,50-60 min,Best meal I've had in a while!
4,5,Tech Park,Mumbai,200,4.7,6926,Spring Rolls,Gluten-Free,20-30 min,Mediocre experience.
...,...,...,...,...,...,...,...,...,...,...
7995,7996,City Center,Mumbai,300,4.0,3303,BBQ Ribs,Vegan,20-30 min,My new favorite dish!
7996,7997,Downtown,Chennai,100,4.7,8742,Butter Chicken,Non-Vegetarian,20-30 min,Amazing taste and quick delivery.
7997,7998,Tech Park,Chennai,900,4.5,4645,Mango Shake,Fast Food,30-40 min,Nothing special but edible.
7998,7999,Old Town,Delhi,500,4.2,3218,Grilled Cheese,Non-Vegetarian,50-60 min,It was okay.


In [31]:
df=data.loc[:,["Review","Avg Rating"]]

In [32]:
df

,Review,Avg Rating
0,"Good, but nothing extraordinary.",4.2
1,"Good, but nothing extraordinary.",4.7
2,Late delivery ruined it.,4.7
3,Best meal I've had in a while!,4.0
4,Mediocre experience.,4.7
...,...,...
7995,My new favorite dish!,4.0
7996,Amazing taste and quick delivery.,4.7
7997,Nothing special but edible.,4.5
7998,It was okay.,4.2


In [33]:
df["Review"]=df["Review"].apply(lambda x:x.lower())
df["Avg Rating"]=df["Avg Rating"].apply(lambda x:1 if x>3.5 else 0)

In [34]:
df["Avg Rating"].unique()

array([1, 0], dtype=int64)

In [30]:

tokanizer=Tokenizer(num_words=200,oov_token="<OOV>")

In [41]:
tokanizer.fit_on_texts(df["Review"])
inde=tokanizer.index_word
sequence=tokanizer.texts_to_sequences(df["Review"])
sequence

[[49, 11, 12, 50],
 [49, 11, 12, 50],
 [73, 10, 74, 4],
 [34, 5, 35, 36, 37, 6, 38],
 [59, 31],
 [34, 5, 35, 36, 37, 6, 38],
 [29, 3, 2, 30, 10],
 [23, 24, 25, 26],
 [39, 40, 4],
 [20, 21, 2, 22],
 [39, 40, 4],
 [63],
 [56, 3],
 [56, 3],
 [64, 31, 65],
 [59, 31],
 [63],
 [13, 14, 2, 15, 16],
 [63],
 [41, 67, 68],
 [51, 27, 28, 52, 53],
 [49, 11, 12, 50],
 [17, 2, 7, 8, 9],
 [12, 42, 11, 43],
 [39, 40, 4],
 [13, 14, 2, 15, 16],
 [39, 40, 4],
 [72, 3],
 [18, 19],
 [18, 19],
 [54, 55],
 [23, 24, 25, 26],
 [17, 2, 7, 8, 9],
 [12, 42, 11, 43],
 [12, 42, 11, 43],
 [59, 31],
 [17, 2, 7, 8, 9],
 [51, 27, 28, 52, 53],
 [20, 21, 2, 22],
 [32, 2, 33],
 [20, 21, 2, 22],
 [66, 27, 28],
 [13, 14, 2, 15, 16],
 [32, 2, 33],
 [12, 42, 11, 43],
 [64, 31, 65],
 [20, 21, 2, 22],
 [34, 5, 35, 36, 37, 6, 38],
 [18, 19],
 [23, 24, 25, 26],
 [66, 27, 28],
 [54, 55],
 [6, 44, 5],
 [23, 24, 25, 26],
 [39, 40, 4],
 [56, 3],
 [29, 3, 2, 30, 10],
 [6, 44, 5],
 [32, 2, 33],
 [49, 11, 12, 50],
 [49, 11, 12, 50],
 [1

In [44]:
sequence=pad_sequences(sequence,maxlen=10,padding="post")

In [52]:
model=Sequential([
    Embedding(input_dim=200,output_dim=16,input_length=10),
    SimpleRNN(16,return_sequences=False),
    Dense(8,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

c:\Users\Kinzy\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [57]:
xtrain,xtest,ytrain,ytest=train_test_split(sequence,df["Avg Rating"],test_size=0.2,random_state=1)
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.1, random_state=42, stratify=ytrain)


In [58]:
model.fit(
    xtrain, ytrain,
    epochs=20,
    batch_size=32,
    validation_data=(xval, yval),
    verbose=1

)

Epoch 1/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6411 - loss: 0.6534 - val_accuracy: 0.7125 - val_loss: 0.6019
Epoch 2/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7176 - loss: 0.5949 - val_accuracy: 0.7125 - val_loss: 0.6014
Epoch 3/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7100 - loss: 0.6020 - val_accuracy: 0.7125 - val_loss: 0.6031
Epoch 4/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7201 - loss: 0.5922 - val_accuracy: 0.7125 - val_loss: 0.6021
Epoch 5/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7153 - loss: 0.5969 - val_accuracy: 0.7125 - val_loss: 0.6018
Epoch 6/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7118 - loss: 0.6000 - val_accuracy: 0.7125 - val_loss: 0.6021
Epoch 7/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7146 - loss: 0.5970 - val_accuracy: 0.7125 - val_loss: 0.6017
Epoch 8/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7087 - loss: 0.6023 - val_accuracy: 0.

In [59]:
score = model.evaluate(xtest, ytest, verbose=0)

In [61]:
score[1]

0.731249988079071